# DAM

In [ ]:
#!pip show fastai
#!pip install fastai --upgrade
#!pip install libauc

In [ ]:
from fastai.vision.all import *
import pandas as pd
import sys
sys.path.insert(0,'/content/chexpert/models/')
import cam
import util

In [ ]:
base_model_name = 'xse_resnext50-chexpert-final'

dls, labels = util.chexpert_data_loader(bs=8)

In [ ]:
# Load the base model
model_base = cnn_learner(dls, xse_resnext50, n_out=len(labels), path=Path('/content/chexpert/saves/'), y_range=[0,1])
model_base.load(base_model_name)

# Create a new densenet121 model
model_dam = cnn_learner(dls, xse_resnext50, n_out=len(labels), y_range=[0,1]).model

# Replace the body of new model with the body from the base model
base_model_body = model_base.model[0].state_dict()
model_dam[0].load_state_dict(base_model_body)

# Set all parameters to require grad
for param in model_dam.parameters():
    param.requires_grad = True
    
# Initialize DAM learner
dam_learner = util.DAM(model_dam, dls, folder=Path('/content/chexpert/saves/models'))

In [ ]:
print(f'Initial AUC score: {dam_learner.eval()}')

Initial AUC score: 0.5064225515479093


In [11]:
dam_learner.train(max_epoch=4, lr_div=1.5)

epoch: 0, train_loss: -0.016398, train_auc:0.565837, test_auc:0.603565, lr:0.100000
Update Regularizer @ T=22364!
epoch: 1, train_loss: -0.005195, train_auc:0.550360, test_auc:0.632795, lr:0.066667
Update Regularizer @ T=22364!
epoch: 2, train_loss: -0.009628, train_auc:0.558004, test_auc:0.614033, lr:0.044444
Update Regularizer @ T=22364!
epoch: 3, train_loss: -0.003586, train_auc:0.556561, test_auc:0.582059, lr:0.029630
Update Regularizer @ T=22364!
